# BEE 4750 Homework 2: Dissolved Oxygen

**Name**: Lesedi Kgatla

**ID**: lk535

> **Due Date**
>
> Friday, 09/22/23, 9:00pm

## Overview

### Instructions

This assignment asks you to use a simulation model for dissolved oxygen
to assess the impacts of two wastewater streams, including minimum
treatment levels and the impact of uncertain environmental conditions.
You will also be asked to identify a minimum distance for the addition
of a third discharge stream.

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating project at `c:\Users\lk535\Desktop\Classes\BEE 4750\Homeworks\hw02 - LesediK01\hw02-LesediK01`


   Installed HypergeometricFunctions ─ v0.3.21


   Installed FillArrays ────────────── v1.4.1


   Installed Distributions ─────────── v0.25.98


Precompiling

...

  ✓ DualNumbers


  ✓ Rmath


  ✓ HypergeometricFunctions


  ✓ StatsFuns


  ✓ Distributions
  8 dependencies successfully precompiled in 13 seconds. 142 already precompiled.


In [2]:
using Plots
using LaTeXStrings
using Distributions

## Problems (Total: 40 Points)

A river which flows at 6 km/d is receiving waste discharges from two
sources which are 15 km apart, as shown in
<a href="#fig-river" class="quarto-xref">Figure 1</a>. The oxygen
reaeration rate is 0.55 day<sup>-1</sup>, and the decay rates of CBOD
and NBOD are are 0.55 and 0.25 day<sup>-1</sup>, respectively. The
river’s saturated dissolved oxygen concentration is 10m g/L.

![Figure 1: Schematic of the
system](attachment:figures/river_diagram.png)

### Problem 1 (8 points)

If the characteristics of the river inflow and waste discharges are
given in <a href="#tbl-river" class="quarto-xref">Table 1</a>, write a
Julia model to compute the dissolved oxygen concentration from the first
wastewater discharge to an arbitrary distance `d` km downstream. Use
your model to compute the maximum dissolved oxygen concentration up to
50km downstream and how far downriver this maximum occurs.

|    Parameter     |            River Inflow |         Waste Stream 1 |         Waste Stream 2 |
|:----------------:|------------------------:|-----------------------:|-----------------------:|
|      Inflow      | 100,000 m<sup>3</sup>/d | 10,000 m<sup>3</sup>/d | 15,000 m<sup>3</sup>/d |
| DO Concentration |                7.5 mg/L |                 5 mg/L |                 5 mg/L |
|       CBOD       |                  5 mg/L |                50 mg/L |                45 mg/L |
|       NBOD       |                  5 mg/L |                35 mg/L |                35 mg/L |

Table 1: River inflow and waste stream characteristics for Problem 1.

In [1]:
#compute DO concentration at a given distance
function DO(x, Ci, BO_ws1, BO_ws2, NO_ws1, NO_ws2, U, kc, kn, ka, Cs)
    #parameters
    U = 6  # (km/d)
    kc = 0.35  # (d^-1)
    kn = 0.25  # (d^-1)
    ka = 0.55  # (d^-1)
    Cs = 10.0  # mg/L

    #Initialize  initial values from River Inflow
    Ci = 7.5  # mg/L
    Bi = 5  # mg/L
    Ni = 5  # mg/L

    CO = ((Ci * 100000) + (5 * 10000)) / 110000
    BO = ((Bi * 100000) + (50 * 10000)) / 110000
    NO = ((Ni * 100000) + (35 * 10000)) / 110000

    #modular function to calculate DO conc. 
    function Compute_DO(x, CO, BO, NO)
        α1 = exp(-ka * x / U)
        α2 = (kc / (ka - kc)) * (exp(-kc * x / U) - exp(-ka * x / U))
        α3 = (kn / (ka - kn)) * (exp(-kn * x / U) - exp(-ka * x / U))
        C = Cs * (1 - α1) + (CO * α1) - (BO * α2) - (NO * α3)
        return C
    end

    #Update for iterations based on the waste stream encountered
    if (x > 0.0) && (x <= 15.0)  # the region affected by Waste Stream 1
        C = Compute_DO(x, CO, BO, NO)
    elseif x > 15.0  # After Waste Stream 1 and within the region affected by Waste Stream 2
        α1 = exp(-ka * (x-15) / U) #where x applies, distance has been adjusted to x-15
        α2 = (kc / (ka - kc)) * (exp(-kc * (x-15) / U) - exp(-ka * (x-15) / U))
        α3 = (kn / (ka - kn)) * (exp(-kn * (x-15) / U) - exp(-ka * (x-15) / U))
        C_15 = Cs * (1 - α1) + (CO * α1) - (BO * α2) - (NO * α3)

        CO_mix1 = ((C_15 * 110000) + (5 * 15000)) / 125000
        BO_mix1 = ((BO * exp(-kc * (x-15) / U)) * 110000 + (45 * 15000)) / 125000
        NO_mix1 = ((NO * exp(-kn * (x-15) / U)) *110000 + (35 * 15000)) / 125000
        C = Compute_DO(x - 15, CO_mix1, BO_mix1, NO_mix1)
    return C
    end
end


DO (generic function with 1 method)

In [3]:
x = collect(0:0.1:50.0)
DO_values = [] #empty array to store DO values
for i in length(x)
    C = DO(x[i], 7.5, 50, 45, 35, 35, 6, 0.35, 0.25, 0.55, 10)
    append!(DO_values, C)
end

#find element for the minimum DO and the index for the distance 
min_DO = findmin(DO_values) 
min_d = x[min_DO[2]]

# Display minimum DO and its distance
@show min_DO, min_d


(min_DO, min_d) = ((8.458327259164628, 1), 0.0)


((8.458327259164628, 1), 0.0)

### Problem 2 (4 points)

Use your model to plot the dissolved oxygen concentration in the river
from the first waste stream to 50km downstream. What do you notice?

In [3]:
plot(x, DO_values, xlabel = "Distance (km)", ylabel = "DO (mg/L)", title = "[Dissolved Oxygen] Concentration in River", label ="[DO]")

UndefVarError: UndefVarError: `x` not defined

### Problem 3 (3 points)

Under the assumptions of Problem 1, determine the distance from waste
stream 2 it will take for the dissolved oxygen concentration of the
river to recover to 6 mg/L.

In [9]:
#updated initial values from a distance of 15km 
x = 15
C = DO(x, 7.5, 50, 45, 35, 35, 6, 0.35, 0.25, 0.55, 10)

#Compute distance where DO concentration recovers to 6 mg/L after Waste Stream 2
step = 0.1
DO_target = 6.0
while Dissolved_O < DO_target
    C = DO(x, 7.5, 50, 45, 35, 35, 6, 0.35, 0.25, 0.55, 10)
    x += step
end 
return x

distance = x - 15
println("Distance where DO concentration of the river recovers to 6 mg/L is ", distance, " from waste stream 2")

Distance where DO concentration of the river recovers to 6 mg/L is 0 from waste stream 2


### Problem 4 (5 points)

What is the minimum level of treatment (% removal of organic waste) for
waste stream 2 that will ensure that the dissolved oxygen concentration
never drops below 4 mg/L, assuming that waste stream 1 remains
untreated?

In [14]:
#Since the minimum level of treatment is a known variable, this can be used to determine the minimum level of treatment which will act as the constraint used to infer DO  

#initialize variables 
treatment = 0.0
DO_target = 4
step = 0.1
BO_w2_treated = 45.0 * (1 - treatment / 100)
NO_w2_treated = 35.0 * (1 - treatment / 100) #Assumed NO is treated at the same percentage

while min_DO < DO_target
    treatment += step #iteratively increase treatment by a step until the condition of the min_DO is greater than 4 mg/L is satisfied 
    min_DO = DO(min_d, 7.5, 50, BO_w2_treated, 35, NO_w2_treated, 6, 0.35, 0.25, 0.55, 10)
end
treatment_percentage = treatment * 100

#Compute the new DO applying the same principles from Problem 1
x = collect(0:0.1:50.0)
DO_v2 = [] #empty array to store the new DO values

for i in length(x)
    C = DO(x[i], 7.5, 50, BO_w2_treated, 35, NO_w2_treated, 6, 0.35, 0.25, 0.55, 10)
    append!(DO_v2, C)
end

#find element for the minimum DO and the index for the distance 
min_DO = findmin(DO_v2) 
min_d = x[min_DO[2]]

# Display minimum DO, its distance, and the minimum level of treatment (%)
@show min_DO, min_d, treatment_percentage

MethodError: MethodError: no method matching isless(::Tuple{Float64, Int64}, ::Int64)

Closest candidates are:
  isless(!Matched::Union{StatsBase.PValue, StatsBase.TestStat}, ::Real)
   @ StatsBase C:\Users\lk535\.julia\packages\StatsBase\iMkPf\src\statmodels.jl:90
  isless(!Matched::DualNumbers.Dual{<:Real}, ::Real)
   @ DualNumbers C:\Users\lk535\.julia\packages\DualNumbers\5knFX\src\dual.jl:183
  isless(!Matched::AbstractGray, ::Real)
   @ ColorTypes C:\Users\lk535\.julia\packages\ColorTypes\1dGw6\src\operations.jl:38
  ...


### Problem 5 (5 points)

If both waste streams are treated equally, what is the minimum level of
treatment (% removal of organic waste) for the two sources required to
ensure that the dissolved oxygen concentration never drops below 4 mg/L?

In [18]:
# Find minimum treatment percentage to ensure DO does not drop below 4 mg/L for both waste streams

#initialize variables 
treatment = 0.0
DO_target = 4
step = 0.1
CO_w1_treated = 50.0 * (1 - treatment)
NO_w1_treated = 35.0 * (1 - treatment) #Assumed NO is treated at the same percentage
BO_w2_treated = 45.0 * (1 - treatment)
NO_w2_treated = 35.0 * (1 - treatment) #Assumed NO is treated at the same percentage

while min_DO < DO_target
    treatment += step #iteratively increase treatment by a step until the condition of the min_DO is greater than 4 mg/L is satisfied 
    min_DO = DO(min_d, 7.5, 50, BO_w2_treated, NO_w1_treated, BO_w2_treated, NO_w2_treated, 6, 0.25, 0.55, 10)
end
treatment_percentage = treatment * 100

#Compute the new DO applying the same principles from Problem 1
x = collect(0:0.1:50.0)
DO_v2 = [] #empty array to store the new DO values

for i in length(x)
    C = DO(x[i], 7.5, 50, BO_w2_treated, NO_w1_treated, BO_w2_treated, NO_w2_treated, 6, 0.25, 0.55, 10)
    append!(DO_v2, C)
end

#find element for the minimum DO and the index for the distance 
min_DO = findmin(DO_v2) 
min_d = x[min_DO[2]]

# Display minimum DO, its distance, and the minimum level of treatment (%)
@show min_DO, min_d, treatment_percentage

MethodError: MethodError: no method matching isless(::Tuple{Float64, Int64}, ::Int64)

Closest candidates are:
  isless(!Matched::Union{StatsBase.PValue, StatsBase.TestStat}, ::Real)
   @ StatsBase C:\Users\lk535\.julia\packages\StatsBase\iMkPf\src\statmodels.jl:90
  isless(!Matched::DualNumbers.Dual{<:Real}, ::Real)
   @ DualNumbers C:\Users\lk535\.julia\packages\DualNumbers\5knFX\src\dual.jl:183
  isless(!Matched::AbstractGray, ::Real)
   @ ColorTypes C:\Users\lk535\.julia\packages\ColorTypes\1dGw6\src\operations.jl:38
  ...


### Problem 6 (5 points)

Suppose you are responsible for designing a waste treatment plan for
discharges into the river, with a regulatory mandate to keep the
dissolved oxygen concentration above 4 mg/L. Discuss whether you’d opt
to treat waste stream 2 alone or both waste streams equally. What other
information might you need to make a conclusion, if any?

Given consideration, I would opt to treat both waste streams equally. This is because in practice, it would be more efficient and cost-effective to treat a smaller volume of waste more intensively than to treat a larger volume less intensively. Dual treatment would also result in reduced organic loads from both waste sources, resulting in a secondary environmental benefit most likely favourable to the river's ecosystem. In tandem, a dual treatment system also offers a redundancy that provides relability of treatment in the event of a failure in treating one stream. 

A combination of technical, economic, and socio-environmental factors would have to be provided as additional information since inter-variable trade offs are likely to arise, thus impacting decison making. These include extensive cost analyses for the waste treatment plan, an evaluation or review of the efficacy and scalability of the technology deployed in the waste treatment plan - this would be favourable in future proofing the capacity of the waste treatment plan in the likely event of increased organic loads from the waste streams. In addition, input from stakeholders as this will give insight to the concerns pertaining to the ecological services of interest from the river ecosystem, which may be evidenced to be impacted by the waste discharge activity.

### Problem 7 (5 points)

Suppose that it is known that the DO concentrations at the river inflow
can vary uniformly between 6 mg/L and 8 mg/L. How often will the
treatment plan identified in Problem 5 (both waste streams treated
equally) fail to comply with the regulatory standard?

In [22]:
#Compute function using monte_carlo_simulation 
Random.seed!(10)

function P5_feed(treatment_level, Ci)
    #initialize variables 
    treatment = 0.0
    DO_target = 4
    step = 0.1
    CO_w1_treated = 50.0 * (1 - treatment)
    NO_w1_treated = 35.0 * (1 - treatment) #Assumed NO is treated at the same percentage
    BO_w2_treated = 45.0 * (1 - treatment)
    NO_w2_treated = 35.0 * (1 - treatment) #Assumed NO is treated at the same percentage

    while min_DO < DO_target
        treatment += step #iteratively increase treatment by a step until the condition of the min_DO is greater than 4 mg/L is satisfied 
        min_DO = DO(min_d, 7.5, 50, BO_w2_treated, NO_w1_treated, BO_w2_treated, NO_w2_treated, 6, 0.25, 0.55, 10)
    end
    treatment_percentage = treatment * 100

    #Compute the new DO applying the same principles from Problem 1
    x = collect(0:0.1:50.0)
    DO_v2 = [] #empty array to store the new DO values

    for i in length(x)
        C = DO(x[i], 7.5, 50, BO_w2_treated, NO_w1_treated, BO_w2_treated, NO_w2_treated, 6, 0.25, 0.55, 10)
        append!(DO_v2, C)
    end
    return min_DO
end

for trial in 1:trials
    sample = 6 + (2*rand()) #randomly sample DO concentration from uniform distribution between 6 and 8 mg/L
        #Simulate with the sample and the treatment level
        min_DO = P5_feed(treatment, Ci)

        #Check for failure
        if min_DO < 4.0
            failures += 1
        end
    end
    failure_rate = failures / trials
    return failure_rate
end

#Run imulation
trials = 10000  # Choose an appropriate number for desired accuracy
P5_treatment = P5_feed()
failure_rate = monte_carlo_simulation(trials, P5_feed)

@show (failure_rate * 100)

UndefVarError: UndefVarError: `Random` not defined

### Problem 8 (5 points)

A factory is planning a third wastewater discharge into the river
downstream of the second plant. This discharge would consist of 5
m<sup>3</sup>/day of wastewater with a dissolved oxygen content of 4.5
mg/L and CBOD and NBOD levels of 50 and 45 mg/L, respectively.

Assume that the treatment plan you identified in Problem 5 is still in
place for the existing discharges. If the third discharge will not be
treated, under the original inflow conditions (7.5 mg/L DO), how far
downstream from the second discharge does this third discharge need to
be placed to keep the river concentration from dropping below 4 mg/L?

In [23]:
#Initialize parameters
U = 6  # (km/d)
kc = 0.35  # (d^-1)
kn = 0.25  # (d^-1)
ka = 0.55  # (d^-1)
Cs = 10.0  # mg/L

#Initialize inflows
Ci = 7.5  # mg/L
Bi = 5  # mg/L
Ni = 5  # mg/L

BO_w1_treated = 50.0 * (1 - treatment)
NO_w1_treated = 35.0 * (1 - treatment)
BO_w2_treated = 45.0 * (1 - treatment)
NO_w2_treated = 35.0 * (1 - treatment)

function DO_3(x, Ci, BO_w1_treated, NO_w1_treated, BO_w2_treated, NO_w2_treated, w3_Ci, BO_ws3, NO_ws3, U, kc, kn, ka, Cs)
    CO = ((Ci * 100000) + (5 * 10000)) / 110000
    BO = ((Bi * 100000) + (50 * 10000)) / 110000
    NO = ((Ni * 100000) + (35 * 10000)) / 110000

    x3 = x - 15

    α1 = exp(-ka * x3 / U)
    α2 = (kc / (ka - kc)) * (exp(-kc * x3 / U) - exp(-ka * x3 / U))
    α3 = (kn / (ka - kn)) * (exp(-kn * x3 / U) - exp(-ka * x3 / U))
    
    CO_mix = ((DO(x, 7.5, 50, BO_w2_treated, NO_w1_treated, BO_w2_treated, NO_w2_treated, 6, 0.25, 0.55, 10) * 110000) + (5 * 15000)) / 125000
    BO_mix = ((BO * exp(-kc * x / U)) * 125000 + (45 * 15000)) / 130000
    NO_mix = ((NO * exp(-kn * x / U)) *125000 + (35 * 15000)) / 130000
    C = Cs * (1 - α1) + (CO_mix * α1) - (BO_mix * α2) - (NO_mix * α3)
    return C
end

x = collect(0:0.1:15.0)
DO_values1 = [] #empty array to store DO values
for i in length(x)
    C = DO(x[i], 7.5, (50.0 * (1 - treatment)), (35.0 * (1 - treatment)), (45.0 * (1 - treatment)), (35.0 * (1 - treatment)), 4.5, 50, 45, 6, 0.25, 0.55, 10)
    append!(DO_values1, C)
end

x = collect(15.01:0.1:100)
DO_values2 = [] #empty array to store DO values
for i in length(x)
    C = DO(x[i], 7.5, (50.0 * (1 - treatment)), (35.0 * (1 - treatment)), (45.0 * (1 - treatment)), (35.0 * (1 - treatment)), 4.5, 50, 45, 6, 0.25, 0.55, 10)
    append!(DO_values2, C)
end

x = collect(0:0.1:100)
DO_aggr = vcat(DO_values1, DO_values2)

plot(x, DO_aggr, xlabel = "Distance downsteam (km)", ylabel = "DO (mg/L)", title = "[Dissolved Oxygen] Concentration in River", label ="[DO]")


MethodError: MethodError: no method matching DO(::Float64, ::Float64, ::Float64, ::Float64, ::Float64, ::Float64, ::Float64, ::Int64, ::Int64, ::Int64, ::Float64, ::Float64, ::Int64)

Closest candidates are:
  DO(::Any, ::Any, ::Any, ::Any, ::Any, ::Any, ::Any, ::Any, ::Any, ::Any, ::Any)
   @ Main c:\Users\lk535\Desktop\Classes\BEE 4750\Homeworks\hw02 - LesediK01\hw02-LesediK01\hw02.ipynb:2
  DO(::Any, ::Any, ::Any, ::Any)
   @ Main c:\Users\lk535\Desktop\Classes\BEE 4750\Homeworks\hw02 - LesediK01\hw02-LesediK01\hw02.ipynb:8


## References

List any external references consulted, including classmates.

Consulted with Akshara and Christine 

GeeksforGeeks. 2021. Getting an array of all items of a collection in Julia – collect() Method (online) https://www.geeksforgeeks.org/getting-an-array-of-all-items-of-a-collection-in-julia-collect-method/

GeeksforGeeks. 2021. Getting minimum element along with its index in Julia – findmin() Method (online) https://www.geeksforgeeks.org/getting-minimum-element-along-with-its-index-in-julia-findmin-method/ 

GeeksforGeeks. 2020. Concatenation of arrays in Julia – cat(), vcat(), hcat() and hvcat() Methods (online) https://www.geeksforgeeks.org/concatenation-of-arrays-in-julia-cat-vcat-hcat-and-hvcat-methods/

Julia DataFrames. n.d. Functions https://dataframes.juliadata.org/stable/lib/functions/#Base.vcat

StackOverFlow. 2022. Append! vs Push! in Julia. (online) https://stackoverflow.com/questions/72916749/append-vs-push-in-julia#:~:text=About%20push!%2C%20the%20doc%20says,of%20another%20collection%20to%20it.%22

StackOverFlow. 2016. Concatenating arrays in Julia (online) https://stackoverflow.com/questions/39586830/concatenating-arrays-in-julia

StackOverFlow. 2017. Using increments in for loop (online) https://stackoverflow.com/questions/45494129/using-increments-in-a-for-loop